# Convolutional Neural Network
1. MLP의 한계 <br />
이미지 등을 처리할 때, MLP는 일렬로 된 벡터를 처리하게 된다. 이때 두 가지 문제가 발생한다. <br />
첫 번째. 이미지의 공간 정보가 사라진다.<br />
두 번째. 몇 가지 픽셀 값만 바뀌어도, 단순히 평행이동만 한 거여도 결과가 크게 달라진다.<br />

  그래서 kernel, 혹은 filter로 이미지와 convolution 하는 방법을 사용한다.

## 1. 간단한 CNN 모델 만들기

In [1]:
import torch
import torch.nn as nn

In [9]:
inputs = torch.Tensor(1, 1, 28, 28)   # 1개, 1채널, 28x28

conv1 = nn.Conv2d(1, 32, 3, padding=1)                # 입력 1채널, 출력 32채널, 커널 크기 3, 패딩 1
conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)   # 입력 32채널, 출력 32채널, 커널 크기 3, 패딩 1
pool = nn.MaxPool2d(2)    # 커널 사이즈랑 stride 모두 2

print('최초', inputs.shape)                  # [1, 1, 28, 28]
out = conv1(inputs)
print('첫 번째 합성망 통과', out.shape)      # [1, 32, 28, 28]   패딩으로 크기 보존됐고 출력 채널은 32로 설정했었다. (그러면 커널이 32종류인건가?)
out = pool(out)
print('풀링', out.shape)                     # [1, 32, 14, 14]   pooling으로 크기 절반으로 줄었다.
out = conv2(out)
print('두 번째 합성망 통과', out.shape)      # [1, 64, 14, 14]   패딩으로 크기 보존, 출력은 64개 채널
out = pool(out)
print('풀링', out.shape)                     # [1, 64,  7,  7]   pooling으로 크기 절반으로 줄었다.


out = out.view(out.size(0), -1)              # 배치 차원 빼고 일자로 펼친다.
print('일자로 펼치기', out.shape)
fullyConnect = nn.Linear(out.shape[1], 10)   # out을 (배치,10) 벡터로 바꾸는 Fully Connected 층
out = fullyConnect(out)
print('전결망 통과', out.shape)



최초 torch.Size([1, 1, 28, 28])
첫 번째 합성망 통과 torch.Size([1, 32, 28, 28])
풀링 torch.Size([1, 32, 14, 14])
두 번째 합성망 통과 torch.Size([1, 64, 14, 14])
풀링 torch.Size([1, 64, 7, 7])
일자로 펼치기 torch.Size([1, 3136])
전결망 통과 torch.Size([1, 10])


##2. CNN으로 MNIST 분류하기

#### 2.1 초기 세팅

In [4]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)

if device == 'cuda':
    torch.cuda.manual_seed_all(777)

####2.2 상수 설정 및 데이터 로드

In [5]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

mnist_train = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                          train=True, # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(), # 텐서로 변환
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                         train=False, # False를 지정하면 테스트 데이터로 다운로드
                         transform=transforms.ToTensor(), # 텐서로 변환
                         download=True)
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

####2.3 class로 CNN model 설계

In [ ]:
class CNN(torch.nn.Module) :
  def __init__(self) :
    super(CNN, self).__init__()

    self.layer1 = torch.nn.Sequential(
          torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
          torch.nn.ReLU(),
          torch.nn.MaxPool2d(2)
    )
    self.layer2 = torch.nn.Sequential(
          torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
          torch.nn.ReLU(),
          torch.nn.MaxPool2d(2)
    )
    self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)